In [ ]:
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 20.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system 

In [ ]:
import pandas as pd
import numpy as np
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
from transformers import T5Tokenizer, T5ForConditionalGeneration

from datasets import load_dataset
from sklearn.metrics import f1_score
from google.colab import drive
import torch
import nltk
from nltk.translate.bleu_score import sentence_bleu
from nltk.metrics.distance import edit_distance
from textblob import TextBlob
import io

In [ ]:
nltk.download('punkt')

# Mount Google Drive
drive.mount('/content/drive')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Mounted at /content/drive


In [ ]:
# 1. Load dataset and initialize models
dataset = load_dataset("jpwahle/autoregressive-paraphrase-dataset")
roberta_tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
roberta_model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2)
t5_tokenizer = T5Tokenizer.from_pretrained('t5-small')  # Use t5-small for faster training data prep
t5_model = T5ForConditionalGeneration.from_pretrained('t5-small')

# Move T5 model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
t5_model.to(device)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/2.97k [00:00<?, ?B/s]

dataset.tsv:   0%|          | 0.00/163M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/261990 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [ ]:
# Inspect dataset
#print("Dataset features:", dataset['train'].features)
#print("Sample data:", dataset['train'][0])

# Generate synthetic paraphrases for training (batched)
def generate_paraphrase(texts):
    input_texts = [f"paraphrase: {text}" for text in texts]
    inputs = t5_tokenizer(input_texts, return_tensors='pt', max_length=512, truncation=True, padding=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    outputs = t5_model.generate(inputs['input_ids'], max_length=512, num_beams=4, num_return_sequences=1)
    return [t5_tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

# Create paired dataset
def create_paired_dataset(data, num_samples=1000, batch_size=8):
    paired_data = {'text': [], 'paraphrase': [], 'label': []}
    texts = []
    for item in data.select(range(num_samples)):
        text = item['text']
        if isinstance(text, str) and text.strip():
            texts.append(text)
            if len(texts) == batch_size:
                paraphrases = generate_paraphrase(texts)
                for orig, para in zip(texts, paraphrases):
                    paired_data['text'].append(orig)
                    paired_data['paraphrase'].append(para)
                    paired_data['label'].append(1)  # Positive
                    paired_data['text'].append(orig)
                    paired_data['paraphrase'].append(orig)
                    paired_data['label'].append(0)  # Negative
                texts = []
    # Process remaining texts
    if texts:
        paraphrases = generate_paraphrase(texts)
        for orig, para in zip(texts, paraphrases):
            paired_data['text'].append(orig)
            paired_data['paraphrase'].append(para)
            paired_data['label'].append(1)
            paired_data['text'].append(orig)
            paired_data['paraphrase'].append(orig)
            paired_data['label'].append(0)
    return paired_data

train_data = create_paired_dataset(dataset['train'])

# Preprocess paired dataset
def preprocess_data(data, max_length=128):
    encodings = roberta_tokenizer(
        data['text'],
        data['paraphrase'],
        truncation=True,
        padding=True,
        max_length=max_length,
        return_tensors='pt'
    )
    labels = torch.tensor(data['label'])
    return encodings, labels

train_encodings, train_labels = preprocess_data(train_data)

# Create PyTorch Dataset
class ParaphraseDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item
    def __len__(self):
        return len(self.labels)

train_dataset = ParaphraseDataset(train_encodings, train_labels)


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

In [ ]:
# Training arguments (disable hub and WandB)
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,
    per_device_train_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    push_to_hub=False,  # Explicitly disable Hub
    report_to='none',  # Disable WandB and other logging
)

# Initialize Trainer
trainer = Trainer(
    model=roberta_model,
    args=training_args,
    train_dataset=train_dataset,
    compute_metrics=lambda p: {'f1': f1_score(p.label_ids, np.argmax(p.predictions, axis=1))},
)

# Fine-tune model
trainer.train()

# Save fine-tuned model
roberta_model.save_pretrained('/content/drive/MyDrive/roberta_finetuned')
roberta_tokenizer.save_pretrained('/content/drive/MyDrive/roberta_finetuned')


Step,Training Loss
10,0.699100
20,0.678600
30,0.682500
40,0.700700
50,0.686500
60,0.685300
70,0.679100
80,0.669900
90,0.603800
100,0.515000


('/content/drive/MyDrive/roberta_finetuned/tokenizer_config.json',
 '/content/drive/MyDrive/roberta_finetuned/special_tokens_map.json',
 '/content/drive/MyDrive/roberta_finetuned/vocab.json',
 '/content/drive/MyDrive/roberta_finetuned/merges.txt',
 '/content/drive/MyDrive/roberta_finetuned/added_tokens.json')

In [ ]:


# 2. Load 100 sentences from embedded CSV
sentences_csv = """
sentence,source
"The solar system consists of the Sun and the objects that orbit it, including eight planets.","Wikipedia"
"Quantum computing leverages quantum mechanics to perform computations at unprecedented speeds.","arXiv"
"The impact of social media on mental health has been a growing concern in recent studies.","Student Thesis"
"Photosynthesis is the process by which green plants produce energy from sunlight.","Wikipedia"
"Machine learning models require large datasets to achieve high accuracy.","arXiv"
"Effective leadership is crucial for organizational success in dynamic markets.","Student Thesis"
"The theory of relativity revolutionized our understanding of space and time.","Wikipedia"
"Neural networks are inspired by the structure of the human brain.","arXiv"
"Climate change poses significant challenges to global food security.","Student Thesis"
"The periodic table organizes elements based on their atomic number.","Wikipedia"
"Deep learning has transformed image recognition tasks in computer vision.","arXiv"
"Workplace diversity enhances creativity and decision-making processes.","Student Thesis"
"The Earth's atmosphere protects life by filtering harmful solar radiation.","Wikipedia"
"Reinforcement learning enables agents to learn optimal behaviors through rewards.","arXiv"
"Sustainable urban planning is essential for reducing carbon footprints.","Student Thesis"
"The human genome project mapped the entire DNA sequence of humans.","Wikipedia"
"Graph neural networks model relationships in structured data effectively.","arXiv"
"Corporate social responsibility influences consumer trust and loyalty.","Student Thesis"
"Plate tectonics explains the movement of the Earth's lithospheric plates.","Wikipedia"
"Federated learning allows model training without sharing sensitive data.","arXiv"
"Gender equality in education promotes economic development globally.","Student Thesis"
"The Milky Way is our galaxy, containing over 100 billion stars.","Wikipedia"
"Natural language processing enables machines to understand human language.","arXiv"
"Renewable energy adoption is critical for mitigating climate change.","Student Thesis"
"DNA replication is a fundamental process for cell division in organisms.","Wikipedia"
"Attention mechanisms improve the performance of transformer models.","arXiv"
"Employee engagement drives productivity and reduces turnover rates.","Student Thesis"
"The greenhouse effect regulates Earth's temperature but is intensified by emissions.","Wikipedia"
"Blockchain technology ensures secure and transparent transactions.","arXiv"
"Access to clean water is a fundamental human right and global challenge.","Student Thesis"
"Black holes are regions of spacetime with intense gravitational pull.","Wikipedia"
"Generative adversarial networks create realistic synthetic data.","arXiv"
"Ethical leadership fosters trust and integrity within organizations.","Student Thesis"
"The water cycle describes the continuous movement of water on Earth.","Wikipedia"
"Transfer learning accelerates model training on new tasks.","arXiv"
"Financial literacy empowers individuals to make informed economic decisions.","Student Thesis"
"The Hubble Space Telescope has provided unprecedented views of the universe.","Wikipedia"
"Convolutional neural networks excel in processing grid-like data.","arXiv"
"Public health campaigns reduce the spread of infectious diseases.","Student Thesis"
"The Pythagorean theorem relates the sides of a right triangle.","Wikipedia"
"Autoencoders are used for unsupervised learning and data compression.","arXiv"
"Cultural competence improves healthcare outcomes in diverse populations.","Student Thesis"
"The ozone layer absorbs most of the Sun's ultraviolet radiation.","Wikipedia"
"Recurrent neural networks are suited for sequential data processing.","arXiv"
"Corporate governance ensures accountability and transparency in firms.","Student Thesis"
"The Big Bang theory explains the origin of the universe.","Wikipedia"
"Word embeddings capture semantic relationships in text data.","arXiv"
"Social entrepreneurship addresses societal issues through innovative solutions.","Student Thesis"
"Volcanic eruptions can significantly impact global climate patterns.","Wikipedia"
"Self-supervised learning leverages unlabeled data for model training.","arXiv"
"Conflict resolution strategies enhance team collaboration.","Student Thesis"
"The cell membrane regulates what enters and leaves the cell.","Wikipedia"
"Bayesian networks model probabilistic relationships in data.","arXiv"
"Universal basic income could address economic inequality.","Student Thesis"
"The theory of evolution describes how species change over time.","Wikipedia"
"Adversarial attacks exploit vulnerabilities in machine learning models.","arXiv"
"Remote work has reshaped organizational culture and productivity.","Student Thesis"
"The Internet connects billions of devices worldwide.","Wikipedia"
"Sentiment analysis gauges opinions from text data.","arXiv"
"Mental health stigma hinders access to treatment.","Student Thesis"
"The carbon cycle regulates carbon exchange in Earth's systems.","Wikipedia"
"Explainable AI improves trust in machine learning predictions.","arXiv"
"Supply chain resilience is critical for global trade.","Student Thesis"
"The immune system defends the body against pathogens.","Wikipedia"
"Knowledge graphs enhance information retrieval and reasoning.","arXiv"
"Microfinance supports entrepreneurship in developing economies.","Student Thesis"
"The speed of light is a universal constant in physics.","Wikipedia"
"Domain adaptation improves model performance across datasets.","arXiv"
"Corporate sustainability aligns business goals with environmental impact.","Student Thesis"
"The nitrogen cycle is essential for life on Earth.","Wikipedia"
"Active learning optimizes data labeling for model training.","arXiv"
"Work-life balance improves employee well-being and performance.","Student Thesis"
"The Doppler effect describes changes in wave frequency.","Wikipedia"
"Zero-shot learning enables models to generalize to unseen classes.","arXiv"
"Community engagement strengthens local governance.","Student Thesis"
"The human brain contains approximately 86 billion neurons.","Wikipedia"
"Ensemble methods combine multiple models for better predictions.","arXiv"
"Globalization impacts cultural identity and economic systems.","Student Thesis"
"Coral reefs support diverse marine ecosystems.","Wikipedia"
"Probabilistic graphical models represent complex dependencies.","arXiv"
"Digital transformation enhances business efficiency.","Student Thesis"
"The electromagnetic spectrum includes all wavelengths of light.","Wikipedia"
"Meta-learning enables models to learn how to learn.","arXiv"
"Social media influences political polarization.","Student Thesis"
"Antibiotics combat bacterial infections but face resistance.","Wikipedia"
"Multi-task learning improves efficiency in model training.","arXiv"
"Corporate ethics shape public perception and trust.","Student Thesis"
"The jet stream influences weather patterns globally.","Wikipedia"
"Continual learning enables models to adapt to new data.","arXiv"
"Public policy shapes economic and social outcomes.","Student Thesis"
"Photosynthetic organisms produce oxygen for Earth's atmosphere.","Wikipedia"
"Sparse coding models data with minimal active components.","arXiv"
"Financial inclusion promotes economic empowerment.","Student Thesis"
"The Moon's gravity causes tides on Earth.","Wikipedia"
"Neural architecture search automates model design.","arXiv"
"Employee training enhances organizational competitiveness.","Student Thesis"
"""
sentences_df = pd.read_csv(io.StringIO(sentences_csv))
original_sentences = sentences_df['sentence'].tolist()

# Save sentences to Google Drive
sentences_df.to_csv('/content/drive/MyDrive/sentences.csv', index=False)

# Switch to t5-base for final paraphrase generation
t5_tokenizer = T5Tokenizer.from_pretrained('t5-base')
t5_model = T5ForConditionalGeneration.from_pretrained('t5-base').to(device)


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
# Load the watermarked_sentences.csv from Google Drive
file_path = '/content/drive/My Drive/watermarked_sentences.csv'
sentences_df = pd.read_csv(file_path)

# Extract the watermarked_paraphrase column as a list
watermarked_sentences = sentences_df['watermarked_paraphrase'].tolist()

# Save the DataFrame to Google Drive
sentences_df.to_csv('/content/drive/MyDrive/processed_watermarked_sentences.csv', index=False)

# Switch to t5-base for final paraphrase generation
t5_tokenizer = T5Tokenizer.from_pretrained('t5-base')
t5_model = T5ForConditionalGeneration.from_pretrained('t5-base').to(device)

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
# 3. Generate paraphrases for input sentences
paraphrases = []
for sentence in watermarked_sentences:
    input_text = f"paraphrase: {sentence}"
    inputs = t5_tokenizer(input_text, return_tensors='pt', max_length=512, truncation=True).to(device)
    outputs = t5_model.generate(inputs['input_ids'], max_length=512, num_beams=4, num_return_sequences=2)
    para_list = [t5_tokenizer.decode(output, skip_special_tokens=True) for output in outputs]
    paraphrases.append(para_list)
    print(f"\nOriginal: {sentence}")
    print(f"Paraphrase 1: {para_list[0]}")
    if len(para_list) > 1:
        print(f"Paraphrase 2: {para_list[1]}")
    else:
        print("Paraphrase 2: None")



Original: 

The sun's surface reflects light back into space.

The reflection changes the solar system's orbit, as objects like Mars and Mercury move farther away
Paraphrase 1: True
Paraphrase 2: False

Original:  The more powerful the quantum system, the more quickly it can perform computations. However, most quantum systems are
Paraphrase 1: False
Paraphrase 2: True

Original:  In a landmark 2018 study, researchers from the University of California found that people experiencing severe emotional distress from social media use tended to be more sensitive to criticism and more
Paraphrase 1: False
Paraphrase 2: positive

Original:  There are two major pathways by which photosynthetic reactions occur. The first is the photosynthetic pathway through the chlor
Paraphrase 1: False
Paraphrase 2: True

Original: 

Data are typically gathered by manually gathering the data from multiple users.

Most existing ML
Paraphrase 1: True
Paraphrase 2: False

Original:  Companies have to adapt their t

In [ ]:
# 4. Evaluate paraphrases
def evaluate_text(text, original):
    # Compute clarity using BLEU score
    clarity = sentence_bleu([original.split()], text.split())
    # Compute fluency using TextBlob
    fluency = TextBlob(text).sentiment.polarity
    # Compute embeddings for original and paraphrase separately
    orig_inputs = roberta_tokenizer(original, return_tensors='pt', truncation=True, padding=True).to(device)
    text_inputs = roberta_tokenizer(text, return_tensors='pt', truncation=True, padding=True).to(device)
    with torch.no_grad():
        orig_embeddings = roberta_model.roberta(**orig_inputs).last_hidden_state.mean(dim=1)  # Shape: (1, hidden_size)
        text_embeddings = roberta_model.roberta(**text_inputs).last_hidden_state.mean(dim=1)  # Shape: (1, hidden_size)
    # Compute coherence using cosine similarity
    coherence = torch.nn.functional.cosine_similarity(orig_embeddings, text_embeddings, dim=1).item()
    return clarity, fluency, coherence

evaluations = []
for idx, (original, para_list) in enumerate(zip(watermarked_sentences, paraphrases)):
    for para in para_list:
        clarity, fluency, coherence = evaluate_text(para, original)
        evaluations.append({
            'original': original,
            'paraphrase': para,
            'clarity': clarity,
            'fluency': fluency,
            'coherence': coherence
        })
        print(f"\nEvaluation for Original: {original}")
        print(f"Paraphrase: {para}")
        print(f"Clarity: {clarity:.4f}, Fluency: {fluency:.4f}, Coherence: {coherence:.4f}")



Evaluation for Original: 

The sun's surface reflects light back into space.

The reflection changes the solar system's orbit, as objects like Mars and Mercury move farther away
Paraphrase: True
Clarity: 0.0000, Fluency: 0.3500, Coherence: 0.6378

Evaluation for Original: 

The sun's surface reflects light back into space.

The reflection changes the solar system's orbit, as objects like Mars and Mercury move farther away
Paraphrase: False
Clarity: 0.0000, Fluency: -0.4000, Coherence: 0.7003

Evaluation for Original:  The more powerful the quantum system, the more quickly it can perform computations. However, most quantum systems are
Paraphrase: False
Clarity: 0.0000, Fluency: -0.4000, Coherence: 0.5966

Evaluation for Original:  The more powerful the quantum system, the more quickly it can perform computations. However, most quantum systems are
Paraphrase: True
Clarity: 0.0000, Fluency: 0.3500, Coherence: 0.6221

Evaluation for Original:  In a landmark 2018 study, researchers from th

/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_


Evaluation for Original:  In the past, models were trained by each user and the data was shared between different users
Paraphrase: False
Clarity: 0.0000, Fluency: -0.4000, Coherence: 0.6414

Evaluation for Original:  In the past, models were trained by each user and the data was shared between different users
Paraphrase: True
Clarity: 0.0000, Fluency: 0.3500, Coherence: 0.6809

Evaluation for Original: 

Research on education has shown time and time again that the most powerful economic
Paraphrase: : To paraphrase: To paraphrase: To paraphrase: To paraphrase: To paraphrase: To paraphrase: To paraphrase: To paraphrase: To paraphrase: To paraphrase: To paraphrase: To paraphrase: To paraphrase: To paraphrase: To paraphrase: To paraphrase: To paraphrase: To paraphrase: To paraphrase: To paraphrase: To paraphrase: To paraphrase: To paraphrase: To paraphrase: To paraphrase: To paraphrase: To paraphrase
Clarity: 0.0000, Fluency: 0.0000, Coherence: 0.3411

Evaluation for Original: 

Research

In [ ]:
# 5. Plagiarism detection
def roberta_plagiarism_check(original, paraphrase):
    inputs = roberta_tokenizer(original, paraphrase, return_tensors='pt', truncation=True, padding=True).to(device)
    with torch.no_grad():
        logits = roberta_model(**inputs).logits
    return torch.softmax(logits, dim=1)[0][1].item()

def turnitin_check(original, paraphrase):
    edit_dist = edit_distance(original.lower(), paraphrase.lower())
    max_len = max(len(original), len(paraphrase))
    similarity = 1 - (edit_dist / max_len)
    return similarity if similarity > 0.3 else 0

def copyscape_check(original, paraphrase):
    bleu = sentence_bleu([original.split()], paraphrase.split())
    return bleu if bleu > 0.2 else 0

plagiarism_results = []
roberta_scores = []
turnitin_scores = []
copyscape_scores = []
for original, para_list in zip(watermarked_sentences, paraphrases):
    for para in para_list:
        roberta_score = roberta_plagiarism_check(original, para)
        turnitin_score = turnitin_check(original, para)
        copyscape_score = copyscape_check(original, para)
        plagiarism_results.append({
            'original': original,
            'paraphrase': para,
            'roberta_score': roberta_score,
            'turnitin_score': turnitin_score,
            'copyscape_score': copyscape_score
        })
        roberta_scores.append(roberta_score)
        turnitin_scores.append(turnitin_score)
        copyscape_scores.append(copyscape_score)

        # Compute F1 scores
true_labels = [1] * len(plagiarism_results)
roberta_pred = [1 if score > 0.5 else 0 for score in roberta_scores]
turnitin_pred = [1 if score > 0.5 else 0 for score in turnitin_scores]
copyscape_pred = [1 if score > 0.5 else 0 for score in copyscape_scores]

roberta_f1 = f1_score(true_labels, roberta_pred)
turnitin_f1 = f1_score(true_labels, turnitin_pred)
copyscape_f1 = f1_score(true_labels, copyscape_pred)

# Print F1 scores
print(f"\nF1 Scores for Paraphrase Detection:")
print(f"RoBERTa: {roberta_f1:.4f}")
print(f"Turnitin: {turnitin_f1:.4f}")
print(f"Copyscape: {copyscape_f1:.4f}")

# Save F1 scores
with open('/content/drive/MyDrive/f1_scores.txt', 'w') as f:
    f.write(f"RoBERTa F1: {roberta_f1}\nTurnitin F1: {turnitin_f1}\nCopyscape F1: {copyscape_f1}")



F1 Scores for Paraphrase Detection:
RoBERTa: 0.9895
Turnitin: 0.1359
Copyscape: 0.1449
